# U.S. Medical Insurance 
### **Business Intelligence Data Analysis Project**

---
### **Description**

- This notebook is part of a project designed to develop and demonstrate skills in business intelligence and data analysis. The focus is to perform data analysis using Python by implementing key programming and analytical techniques.
---
### **Project Objectives**

#### 1. **Work locally on your own computer**
- All tasks are performed locally, ensuring familiarity with file management and local development tools.

#### 2. **Import a dataset into your program**
- Load data from files (e.g., CSV, Excel, etc.) and prepare it for analysis by addressing any data loading issues.

#### 3. **Analyze a dataset by building out functions or class methods**
- Write reusable functions or classes to clean, process, and extract insights from the data.

#### 4. **Use libraries to assist in your analysis**
- Leverage Python libraries such as `pandas`, `NumPy`, and `matplotlib` for data manipulation, computation, and visualization.
---
### **Optional Objectives**

#### 5. **Document and organize your findings**
- Summarize insights, observations, and results to ensure clarity and potential sharing.

#### 6. **Make predictions about a dataset’s features based on your findings**
- Apply predictive modeling techniques (optional) to make informed projections about dataset trends.
---


In [372]:
import csv  # Importing the CSV module

# Setting Variables
file_path = 'insurance.csv'  # Replace with the path to your CSV file

#################################################################################################################

# Function to load data from a CSV file
def load_csv_data():
    """
    This function reads data from a CSV file and returns a list of dictionaries.
    Each dictionary represents one row in the CSV, with keys as column headers.
    """
    with open(file_path, mode='r') as file:  # Open the file in read mode
        reader = csv.DictReader(file)  # Read the file as a dictionary
        return [row for row in reader]  # Convert the rows to a list of dictionaries

# Function to convert string values in a dictionary to numbers where applicable
def str_to_num(row):
    """
    Converts all numeric strings in a dictionary row to int or float.
    Leaves non-numeric values unchanged.

    Parameters:
    - row: A dictionary representing a single row of the CSV file.
    
    Returns:
    - A cleaned dictionary with numeric values converted.
    """
    for key, value in row.items():  # Loop through each key-value pair
        if value is None or value == '':  # Skip None or empty string values
            row[key] = 0  # Replace missing values with 0
        try:
            # Check if the value is a string and contains a decimal point
            if isinstance(value, str) and '.' in value:
                row[key] = float(value)  # Convert to a float
            elif isinstance(value, str) and value.isdigit():
                row[key] = int(value)  # Convert to an integer
        except ValueError:
            # Leave the value unchanged if it's not convertible
            pass
    return row

# Function to extract values from a specific column
def get_column_values(column_name):
    """
    Extracts all values from a specific column in the CSV file.

    Parameters:
    - column_name: The name of the column to extract values from.

    Returns:
    - A list of values from the specified column, converted to numbers if applicable.
    """
    data = load_csv_data()  # Load data from the CSV file
    data = [str_to_num(row) for row in data]  # Apply str_to_num to clean data
    return [row[column_name] for row in data]  # Extract values for the given column

def count_group_instances(group_dict=None, column=None):
    """
    Counts the instances of members of a group defined by a dictionary or unique column values.

    Parameters:
    - data: List of dictionaries (cleaned data).
    - group_dict: (Optional) A dictionary where keys define the groups (e.g., age ranges),
                  and values provide group descriptions. If None, counts unique values in the column.
    - column: The column in the data to check against the groups.

    Returns:
    - A dictionary where keys are group descriptions or unique values and values are counts.
    """
    if not column:
        raise ValueError("A column must be specified for grouping.")

    group_counts = {}

    if group_dict:  # If a dictionary is provided for grouping
        group_counts = {description: 0 for description in group_dict.values()}  # Initialize group counts

        for row in cleaned_data:
            value = row.get(column)  # Extract the value from the specified column
            if not isinstance(value, (int, float)) and not group_dict:  # Skip invalid or non-numeric values
                continue

            for range_key, description in group_dict.items():
                # Process ranges
                if "-" in range_key:  # Closed range
                    low, high = map(int, range_key.split("-"))
                    if isinstance(value, (int, float)) and low <= value <= high:
                        group_counts[description] += 1
                        break
                elif "+" in range_key:  # Open-ended range
                    low = int(range_key.split("+")[0])
                    if isinstance(value, (int, float)) and value >= low:
                        group_counts[description] += 1
                        break

    else:  # If no group_dict, count unique values in the column
        group_counts = {}
        for row in cleaned_data:
            value = row.get(column)
            if value is None or value == '':
                value = "Unknown"  # Handle missing or empty values
            group_counts[value] = group_counts.get(value, 0) + 1

    return group_counts


###############################################################################################################3

# Process data
data = load_csv_data()  # Load the raw data
cleaned_data = [str_to_num(row) for row in data]  # Clean the data using str_to_num

# Extract information
customer_qty = len(cleaned_data)  # Get the total number of customers
ages_lst = get_column_values('age') # Extract the 'age' column
genders_lst = get_column_values('sex')  # Extract the 'sex' column
bmi = get_column_values('bmi')  # Extract the 'bmi' column
children = get_column_values('children')  # Extract the 'children' column
smoker = get_column_values('smoker')  # Extract the 'smoker' column
region = get_column_values('region')  # Extract the 'region' column
charges = get_column_values('charges')  # Extract the 'charges' column


In [373]:
# Average age calculation
avg_age = sum([age for age in ages_lst ]) / customer_qty  # The mean age of individuals in the dataset

# Define age groups for categorization
age_groups = {
    "0-17": "children/minors",
    "18-24": "young adults",
    "25-34": "early career",
    "35-44": "mid-career adults",
    "45-54": "pre-senior adults",
    "55-64": "early retirees",
    "65+": "Medicare-eligible seniors",
}  # Group ages into bins showing the max age for each group, or minimal age in case of 65+'s

# Calculate age distribution
age_distribution = count_group_instances(age_groups, 'age')

# Find oldest and youngest ages
oldest_age = max([age for age in ages_lst])
youngest_age = min([age for age in ages_lst])

# Format age distribution into a multiline string
age_distribution_lines = "\n".join([
    f"        -{description.title()} ({range_key}): {value}" #prints each group age range in a new line
    for range_key, description in age_groups.items() 
    for key, value in age_distribution.items() 
    if description == key #This filters out mismatched items, if any.
    ])

# Print findings
print(f'''Age-related findings:

    - Average Age: The mean age of individuals in the dataset is {avg_age:.0f} years.
    - Oldest Age: The oldest age in the dataset is {oldest_age} years.
    - Youngest Age: The youngest age in the dataset is {youngest_age} years.

    - Age Distribution by Group: 
{age_distribution_lines}

''')

Age-related findings:

    - Average Age: The mean age of individuals in the dataset is 39 years.
    - Oldest Age: The oldest age in the dataset is 64 years.
    - Youngest Age: The youngest age in the dataset is 18 years.

    - Age Distribution by Group: 
        -Children/Minors (0-17): 0
        -Young Adults (18-24): 278
        -Early Career (25-34): 271
        -Mid-Career Adults (35-44): 260
        -Pre-Senior Adults (45-54): 287
        -Early Retirees (55-64): 242
        -Medicare-Eligible Seniors (65+): 0




In [412]:
gender_dist = count_group_instances(None, 'sex')
avg_charges_by_gender = { 
    key: sum([
        charge for gender, 
        charge in zip(genders_lst, charges) 
        if gender == key
    ]) / gender_dist[key]
    for key in gender_dist
}
print(avg_charges_by_gender)

print(list(zip(genders_lst, smoker)))
    
# '''
# 2. Sex

#     Gender Distribution: Count the number of males and females.
#     Average Charges by Gender: Calculate the mean charges for males and females.
#     Smoker Proportion by Gender: Compare the percentage of smokers within each gender.
# '''

{'female': 12569.578843835347, 'male': 13956.751177721893}
[('female', 'yes'), ('male', 'no'), ('male', 'no'), ('male', 'no'), ('male', 'no'), ('female', 'no'), ('female', 'no'), ('female', 'no'), ('male', 'no'), ('female', 'no'), ('male', 'no'), ('female', 'yes'), ('male', 'no'), ('female', 'no'), ('male', 'yes'), ('male', 'no'), ('female', 'no'), ('male', 'no'), ('male', 'no'), ('male', 'yes'), ('female', 'no'), ('female', 'no'), ('male', 'no'), ('female', 'yes'), ('male', 'no'), ('female', 'no'), ('female', 'no'), ('female', 'no'), ('male', 'no'), ('male', 'yes'), ('male', 'yes'), ('female', 'no'), ('female', 'no'), ('male', 'no'), ('male', 'yes'), ('male', 'no'), ('female', 'no'), ('male', 'no'), ('male', 'yes'), ('male', 'yes'), ('female', 'no'), ('female', 'no'), ('male', 'no'), ('female', 'no'), ('male', 'no'), ('male', 'no'), ('female', 'no'), ('female', 'no'), ('female', 'no'), ('male', 'yes'), ('female', 'no'), ('female', 'no'), ('male', 'yes'), ('male', 'yes'), ('female', 'n

In [413]:

'''
3. BMI (Body Mass Index)

    Average BMI: The mean BMI across all individuals.
    BMI Categories:
        Underweight (BMI < 18.5)
        Normal weight (18.5 ≤ BMI < 25)
        Overweight (25 ≤ BMI < 30)
        Obesity (BMI ≥ 30)
        Count individuals in each category.
    Correlation with Charges: Analyze the relationship between BMI and charges.
'''

'\n3. BMI (Body Mass Index)\n\n    Average BMI: The mean BMI across all individuals.\n    BMI Categories:\n        Underweight (BMI < 18.5)\n        Normal weight (18.5 ≤ BMI < 25)\n        Overweight (25 ≤ BMI < 30)\n        Obesity (BMI ≥ 30)\n        Count individuals in each category.\n    Correlation with Charges: Analyze the relationship between BMI and charges.\n'

In [414]:

'''4. Children

    Average Number of Children: The mean number of children per individual.
    Proportion with No Children: Percentage of individuals without children.
    Average Charges by Number of Children: Analyze how the number of children impacts charges.
'''

'4. Children\n\n    Average Number of Children: The mean number of children per individual.\n    Proportion with No Children: Percentage of individuals without children.\n    Average Charges by Number of Children: Analyze how the number of children impacts charges.\n'

In [415]:

'''
5. Smoker

    Smoker vs. Non-Smoker Proportion: Percentage of smokers and non-smokers.
    Average Charges for Smokers vs. Non-Smokers: Compare the mean charges for both groups.
    Correlation Between Smoking and Region: Check if smokers are concentrated in certain regions.
'''

'\n5. Smoker\n\n    Smoker vs. Non-Smoker Proportion: Percentage of smokers and non-smokers.\n    Average Charges for Smokers vs. Non-Smokers: Compare the mean charges for both groups.\n    Correlation Between Smoking and Region: Check if smokers are concentrated in certain regions.\n'

In [416]:

'''
6. Region

    Population Distribution by Region: Count individuals in each region (e.g., southeast, southwest).
    Average BMI by Region: Analyze the mean BMI for individuals in each region.
    Average Charges by Region: Calculate the mean charges for each region.
'''

'\n6. Region\n\n    Population Distribution by Region: Count individuals in each region (e.g., southeast, southwest).\n    Average BMI by Region: Analyze the mean BMI for individuals in each region.\n    Average Charges by Region: Calculate the mean charges for each region.\n'

In [417]:

'''
7. Charges

    Average Charges: The mean charges across all individuals.
    Median Charges: The median charges, which can give insights into central tendency.
    Highest and Lowest Charges: Identify the maximum and minimum charges in the dataset.
    Charges Distribution: Visualize the charges using a histogram or boxplot.
    Correlation with Other Variables:
        Charges vs. Age
        Charges vs. BMI
        Charges vs. Number of Children
'''

'\n7. Charges\n\n    Average Charges: The mean charges across all individuals.\n    Median Charges: The median charges, which can give insights into central tendency.\n    Highest and Lowest Charges: Identify the maximum and minimum charges in the dataset.\n    Charges Distribution: Visualize the charges using a histogram or boxplot.\n    Correlation with Other Variables:\n        Charges vs. Age\n        Charges vs. BMI\n        Charges vs. Number of Children\n'

In [418]:

'''Cross-Metrics

    Smoker and Region Impact on Charges: Analyze if certain regions have higher charges for smokers.
    Age and BMI Correlation: Check if there's a relationship between age and BMI.
    Impact of Children on Charges by Gender: Investigate how charges vary with the number of children, broken down by gender.
'''


def calculate_proportion_by_group(condition_column, group_column, group_values, condition_value, group_counts):
    """
    Calculates the proportion of a specific condition (e.g., smokers) for each group (e.g., gender).
    
    Parameters:
    - condition_column: List containing the condition values (e.g., ['yes', 'no', 'yes'] for smokers).
    - group_column: List containing the group values (e.g., ['male', 'female', 'male'] for genders).
    - group_values: List of unique group values to calculate proportions for (e.g., ['male', 'female']).
    - condition_value: The value in condition_column to count (e.g., 'yes' for smokers).
    - group_counts: Dictionary containing the total counts of each group (e.g., {'male': 3, 'female': 2}).

    Returns:
    - Dictionary with proportions of the condition for each group.
    """
    proportions = {}
    for group in group_values:
        count = sum(
            1 for group_value, condition in zip(group_column, condition_column)
            if group_value == group and condition == condition_value
        )
        proportions[group] = count / group_counts[group]
    return proportions
